In [123]:
from rdflib import *

In [124]:
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

# Load Schema + Metadata

In [125]:
# Load your RDF ontology file
schema_metadata = Graph()
schema_metadata.parse("../Registration/Source_Graph_DP2.ttl")


<Graph identifier=N0fd1742eb4f942f4b5f80cec28094278 (<class 'rdflib.graph.Graph'>)>

In [126]:
print(schema_metadata.serialize(format='turtle'))

@prefix ab: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix tb: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#> .

ab:dicomimage a <https://www.w3.org/ns/dcat#Dataset> ;
    dcterms:identifier "35f214530372f7ab83043b98c268f58a4c042c35ccb8ed7d45e7b1911b21a659" ;
    tb:hasAttribute ab:AccessionNumber,
        ab:BitsAllocated,
        ab:BitsStored,
        ab:Columns,
        ab:Exposure,
        ab:FrameIncrementPointer,
        ab:FrameTime,
        ab:HighBit,
        ab:ImageType,
        ab:InstanceNumber,
        ab:InstitutionAddress,
        ab:InstitutionName,
        ab:KVP,
        ab:LossyImageCompressionRetired,
        ab:Manufacturer,
        ab:MaskSubtractionSequence,
        ab:Modality,
        ab:NumberOfFrames,
        ab:PatientBirthDate,
        ab:PatientID,
        ab:PatientName,
        ab:PatientOrientation,
        ab:PatientSex,
        ab:PerformingPhysicianNa

## Data Contract

In [127]:
contract = schema_metadata
contract.bind('dqv', "http://www.w3.org/ns/dqv#")

Begin manual contract by adding the schema_metadata to the sdm

In [128]:
from rdflib import *
from hashlib import sha256
dataset_name = 'dicomimage'

### Create Contract

In [129]:
contract.add((abox[f'dc_{dataset_name}'],RDF.type, tbox.DataContract))
contract.add((abox[dataset_name],tbox.hasDC, abox[f'dc_{dataset_name}']))

<Graph identifier=N0fd1742eb4f942f4b5f80cec28094278 (<class 'rdflib.graph.Graph'>)>

### Add Mappings

In [130]:
contract.add((abox.map2, RDF.type, tbox.SchemaMapping))
contract.add((abox[f'dc_{dataset_name}'],tbox.hasMapping, abox.map2))

contract.add((abox.map2, tbox.mfrom, abox.PatientID))
contract.add((abox.map2, tbox.mto, abox.Subject))


<Graph identifier=N0fd1742eb4f942f4b5f80cec28094278 (<class 'rdflib.graph.Graph'>)>

In [131]:
rdf_format = "turtle"  # Choose your desired RDF serialization format
contract.serialize(destination="./SDP2_C.ttl", format=rdf_format)

<Graph identifier=N0fd1742eb4f942f4b5f80cec28094278 (<class 'rdflib.graph.Graph'>)>

### Add Policy

In [132]:
contract.add((abox[f'dc_{dataset_name}'],tbox.hasPolicy, abox.p1))
#contract.add((abox[f'dc_{dataset_name}'],tbox.hasPolicy, abox.p2))
#contract.add((abox[f'dc_{dataset_name}'],tbox.hasPolicy, abox.p3))

<Graph identifier=N0fd1742eb4f942f4b5f80cec28094278 (<class 'rdflib.graph.Graph'>)>

In [133]:
!pip install rdflib_jsonld
import json


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [134]:
def add_jsonld_instances(graph, path):
    # Adds JSON-LD instances to the graph
    with open(path, 'r') as f:
        json_ld_data = json.loads(f.read())
        instances = Graph().parse(data=json_ld_data, format='json-ld')
        graph += instances
    
    return graph

pc1

In [135]:
p1 = '/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/ComputationalCatalogues/p1.json'
g = add_jsonld_instances(contract, p1)

pc2

In [136]:
#p2 = '/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/ComputationalCatalogues/p2.json'
#g = add_jsonld_instances(contract, p2)

p3

In [137]:
#p3 = '/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/ComputationalCatalogues/p3.json'
#g = add_jsonld_instances(contract, p3)

In [138]:
rdf_format = "turtle"  # Choose your desired RDF serialization format
contract.serialize(destination="./SDP2_C.ttl", format=rdf_format)

<Graph identifier=N0fd1742eb4f942f4b5f80cec28094278 (<class 'rdflib.graph.Graph'>)>

## SEND CONTRACT TO SDM

In [139]:
sdm = Graph().parse("/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/SemanticDataModel/sdm.ttl")

In [140]:
sdm += contract

In [141]:
rdf_format = "turtle"  # Choose your desired RDF serialization format
sdm.serialize(destination="/home/acraf/psr/tfm/Fdatavalidation/FederatedComputationalGovernance/SemanticDataModel/sdm.ttl", format=rdf_format)

<Graph identifier=Ne71f33638660472fbaa7a816373bc3a2 (<class 'rdflib.graph.Graph'>)>